In [1]:
import requests
import pandas as pd
import numpy as np

#### 1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [2]:
def get_items():
    items_response = requests.get('https://python.zgulde.net/api/v1/items')
    items_data = items_response.json()
    items = pd.DataFrame(items_data['payload']['items'])
    base_url = 'https://python.zgulde.net'

    items_response2 = requests.get(base_url + items_data['payload']['next_page'])
    items_data2 = items_response2.json()

    items2 = pd.DataFrame(items_data2['payload']['items'])

    items_repsonse3 = requests.get(base_url + items_data2['payload']['next_page'])

    items_data3 = items_repsonse3.json()

    items3 = pd.DataFrame(items_data3['payload']['items'])

    #Add the results to my items dataframe
    items = pd.concat([items,items2, items3]).reset_index()
    items = items.drop(columns='index')
    return items


In [3]:
items = get_items()
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [4]:
items.shape

(50, 6)

#### 2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [5]:
def get_stores():
    stores_response = requests.get('https://python.zgulde.net/api/v1/stores')
    stores_data = stores_response.json()
    stores = pd.DataFrame(stores_data['payload']['stores'])
    return stores

In [6]:
stores = get_stores()
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### 3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [7]:
def get_sales():
    sales_response = requests.get('https://python.zgulde.net/api/v1/sales')
    sales_data = sales_response.json()
    max_page_sales = sales_data['payload']['max_page']

    sales_url = 'https://python.zgulde.net/api/v1/sales?page='

    sales = pd.DataFrame(sales_data['payload']['sales'])

    for i in range(2, max_page_sales + 1):
        response = requests.get(sales_url + str(i))
        json = response.json()
        df = pd.DataFrame(json['payload']['sales'])
        sales = pd.concat([sales, df])
    sales.reset_index(inplace=True)
    return sales

In [8]:
sales = get_sales()
sales.head()

,index,item,sale_amount,sale_date,sale_id,store
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [9]:
#Store the results of my requests to csv files
items.to_csv('items.csv')
stores.to_csv('stores.csv')
sales.to_csv('sales.csv')

### 5. Combine the data from your three separate dataframes into one large dataframe.

In [17]:
#Left merge items onto sales
super_df = sales.merge(items, how='left',
                       left_on='item', right_on='item_id')
super_df.head()

,index,item,sale_amount,sale_date,sale_id,store,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [18]:
#Left merge stores onto my super dataframe
super_df = super_df.merge(stores, how='left',
                          left_on='store', right_on='store_id')
super_df.head()

,index,item,sale_amount,sale_date,sale_id,store,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_id,store_state,store_zipcode
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [19]:
#Looking to see if there are any unesscary columns
super_df.columns

Index(['index', 'item', 'sale_amount', 'sale_date', 'sale_id', 'store',
       'item_brand', 'item_id', 'item_name', 'item_price', 'item_upc12',
       'item_upc14', 'store_address', 'store_city', 'store_id', 'store_state',
       'store_zipcode'],
      dtype='object')

In [20]:
#Drop unnecessary columns from my dataframe
super_df.drop(columns=['index','item', 'sale_id', 'store', 'item_id', 'item_upc12', 'item_upc14',
                       'store_id'], inplace=True)
super_df.head()

,sale_amount,sale_date,item_brand,item_name,item_price,store_address,store_city,store_state,store_zipcode
0,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
2,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
3,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,12125 Alamo Ranch Pkwy,San Antonio,TX,78253
4,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",Riceland,Riceland American Jazmine Rice,0.84,12125 Alamo Ranch Pkwy,San Antonio,TX,78253


6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [21]:
opsd = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

In [22]:
opsd

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714


In [23]:
opsd.to_csv('opsd.csv')

### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.
